In [1]:
import cv2
import mediapipe as mp
import time
import math as math


class HandTrackingDynamic:
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.__mode__   =  mode
        self.__maxHands__   =  maxHands
        self.__detectionCon__   =   detectionCon
        self.__trackCon__   =   trackCon
        self.handsMp = mp.solutions.hands
        self.hands = self.handsMp.Hands()
        self.mpDraw= mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findFingers(self, frame, draw=True):
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)  
        if self.results.multi_hand_landmarks: 
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(frame, handLms,self.handsMp.HAND_CONNECTIONS)

        return frame

    def findPosition( self, frame, handNo=0, draw=True):
        xList =[]
        yList =[]
        bbox = []
        self.lmsList=[]
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
            
                h, w, c = frame.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                self.lmsList.append([id, cx, cy])
                if draw:
                    cv2.circle(frame,  (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
            print( "Hands Keypoint")
            print(bbox)
            if draw:
                cv2.rectangle(frame, (xmin - 20, ymin - 20),(xmax + 20, ymax + 20),
                               (0, 255 , 0) , 2)

        return self.lmsList, bbox
    
    def findFingerUp(self):
         fingers=[]

         if self.lmsList[self.tipIds[0]][1] > self.lmsList[self.tipIds[0]-1][1]:
              fingers.append(1)
         else:
              fingers.append(0)

         for id in range(1, 5):            
              if self.lmsList[self.tipIds[id]][2] < self.lmsList[self.tipIds[id]-2][2]:
                   fingers.append(1)
              else:
                   fingers.append(0)
        
         return fingers

    def findDistance(self, p1, p2, frame, draw= True, r=15, t=3):
         
        x1 , y1 = self.lmsList[p1][1:]
        x2, y2 = self.lmsList[p2][1:]
        cx , cy = (x1+x2)//2 , (y1 + y2)//2

        if draw:
              cv2.line(frame,(x1, y1),(x2,y2) ,(255,0,255), t)
              cv2.circle(frame,(x1,y1),r,(255,0,255),cv2.FILLED)
              cv2.circle(frame,(x2,y2),r, (255,0,0),cv2.FILLED)
              cv2.circle(frame,(cx,cy), r,(0,0.255),cv2.FILLED)
        len= math.hypot(x2-x1,y2-y1)

        return len, frame , [x1, y1, x2, y2, cx, cy]

def main():
        
        ctime=0
        ptime=0
        cap = cv2.VideoCapture(0)
        detector = HandTrackingDynamic()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        if not cap.isOpened():
            print("Cannot open camera")
            exit()

        while True:
            ret, frame = cap.read()

            frame = detector.findFingers(frame)
            lmsList = detector.findPosition(frame)
            # if len(lmsList)!=0:
                #print(lmsList[0])

            ctime = time.time()
            fps =1/(ctime-ptime)
            ptime = ctime

            cv2.putText(frame, str(int(fps)), (10,70), cv2.FONT_HERSHEY_PLAIN,3,(255,0,255),3)
 
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('frame', frame)
            cv2.waitKey(1)


                
if __name__ == "__main__":
            main()

2024-11-08 15:28:02.340558: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-08 15:28:02.564776: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 15:28:02.564899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 15:28:02.593006: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 15:28:02.652529: I tensorflow/core/platform/cpu_feature_guar

Hands Keypoint
(365, 356, 441, 418)
Hands Keypoint
(370, 349, 447, 423)
Hands Keypoint
(370, 350, 448, 425)
Hands Keypoint
(535, 408, 606, 472)
Hands Keypoint
(535, 402, 601, 468)
Hands Keypoint
(536, 405, 605, 469)
Hands Keypoint
(535, 409, 609, 474)
Hands Keypoint
(534, 404, 606, 474)
Hands Keypoint
(533, 402, 604, 472)
Hands Keypoint
(535, 404, 607, 474)
Hands Keypoint
(533, 403, 609, 473)
Hands Keypoint
(535, 403, 608, 475)
Hands Keypoint
(534, 403, 608, 474)
Hands Keypoint
(535, 401, 610, 475)
Hands Keypoint
(535, 400, 610, 477)
Hands Keypoint
(534, 400, 612, 476)
Hands Keypoint
(534, 400, 611, 478)
Hands Keypoint
(532, 396, 612, 475)
Hands Keypoint
(530, 394, 614, 470)
Hands Keypoint
(525, 388, 601, 461)
Hands Keypoint
(518, 381, 606, 451)
Hands Keypoint
(505, 375, 596, 437)
Hands Keypoint
(493, 370, 586, 426)
Hands Keypoint
(475, 370, 563, 416)
Hands Keypoint
(465, 364, 558, 413)
Hands Keypoint
(454, 358, 546, 403)
Hands Keypoint
(447, 355, 539, 399)
Hands Keypoint
(438, 356, 53

KeyboardInterrupt: 